In [29]:
"""
Digital Skeptic - Critical Thinking AI Tool
-------------------------------------------------------------------
Fetches a news article from a given URL and uses Groq's LLM API
to produce a rich, nuanced "Critical Analysis Report" in Markdown.
"""

import sys
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from newspaper import Article
from groq import Groq

# ===============================
# CONFIGURATION
# ===============================

load_dotenv()  # Loads environment variables from a .env file if present

# Grab Groq API key from environment variable (must be set by user)
groq_api_key = os.getenv("GROQ_API_KEY")
if not groq_api_key:
    print("ERROR: Please set your Groq API key via the GROQ_API_KEY environment variable.")
    sys.exit(1)

# Initialize Groq client with your API key for later use
client = Groq(api_key=groq_api_key)

# ===============================
# ARTICLE FETCHING
# ===============================

def fetch_and_clean_article(url):
    """
    Try to extract clean article text and title from a given URL.
    - Uses Newspaper3k library for robust article extraction.
    - If Newspaper3k fails, falls back to BeautifulSoup to pull all <p> paragraph tags.
    Returns:
        (text, title): Tuple of article body text (str) and title (str).
    """
    try:
        # Attempt with Newspaper3k (handles most news sites cleanly)
        article = Article(url)
        article.download()
        article.parse()
        text = article.text
        title = article.title.strip() if article.title else "Untitled"
        if text.strip():  # If extraction was successful and not empty, return
            return text, title
    except Exception as e:
        print(f"[WARN] Newspaper3k failed: {e}")

    # If Newspaper3k didn't work, fallback to BeautifulSoup
    print("[INFO] Falling back to BeautifulSoup extraction...")
    try:
        r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        r.raise_for_status()
        soup = BeautifulSoup(r.text, 'html.parser')
        # Extract all text inside paragraph tags
        paragraphs = [p.get_text() for p in soup.find_all('p')]
        text = "\n".join(paragraphs).strip()
        # Extract article title from <title> tag
        title_tag = soup.find("title")
        title = title_tag.get_text().strip() if title_tag else "Untitled"
        return text, title
    except Exception as e:
        raise RuntimeError(f"Failed to fetch article: {e}")

# ===============================
# CALL GROQ FOR CRITICAL ANALYSIS
# ===============================

def skeptical_analysis_with_llm(article_text, title):
    """
    Sends article text to Groq LLM (Llama 3 70B) for a deep critical analysis.
    Uses a refined prompt for: nuance, subtlety, signs of bias, and logical fallacy identification.
    Returns the generated Markdown report as a string.
    """
    prompt = f"""
You are "The Digital Skeptic" — a critical thinking AI assistant.
Your mission is to help the reader see not only *what* the article says, but *how* it says it.

Analyze the article and produce a **Markdown report** with these sections:

# Critical Analysis Report for: {title}

## 1. Core Claims
- Provide **3–5 bullet points** summarizing the most important factual claims.
- Avoid restating filler details — focus only on key statements that could be challenged or require proof.

## 2. Language & Tone Analysis
- Assess the overall tone (neutral, emotional, persuasive, sensational, etc.).
- Identify any **loaded words, exaggerations, or emotive framing**.
- Comment on whether the language may subtly influence opinions.

## 3. Potential Red Flags
- List **possible signs of bias, spin, gaps, or poor reporting**, such as:
  - Over-reliance on a single source or anonymous sources.
  - Omission of critical counterpoints.
  - Lack of cited data or evidence.
  - Logical fallacies (name the type if spotted: strawman, ad hominem, false cause, etc.).

## 4. Verification Questions
- Suggest **3–4 sharp, specific questions** the reader should investigate to verify the claims.

## 5. Key Entities for Further Research
- List key people, organizations, or places mentioned, and suggest *what about them* should be researched.

## 6. Simulated Opposing Viewpoint
- Write a brief counter-argument as if from someone skeptical of the article's stance.
- Highlight points the author may have ignored.

### Notes:
- Be specific and insightful, not generic.
- Assume the reader is intelligent and wants to think critically.

---ARTICLE START---
{article_text}
---ARTICLE END---
    """

    try:
        response = client.chat.completions.create(
            model="llama3-70b-8192",  # High-quality open model; change if needed
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,         # Low temperature for fact-based repeatability
            max_tokens=2000          # Increase if you expect large output
        )
        return response.choices[0].message.content
    except Exception as e:
        raise RuntimeError(f"Groq API call failed: {e}")

# ===============================
# MAIN EXECUTION
# ===============================

def main():
    """
    Main execution routine:
    - Sets the article URL
    - Fetches and cleans the article
    - Sends the article for Groq-powered skeptical analysis
    - Prints the Markdown report to the console
    """
    # In a production script, get URL from command line or input(). For Colab, hardcode example for demo.
    url = "https://benjaminspall.com/character-traits/"  # Example URL for demonstration

    print(f"[INFO] Fetching article from {url}")
    article_text, title = fetch_and_clean_article(url)

    if not article_text:
        print("ERROR: Could not extract article text.")
        sys.exit(1)

    print("[INFO] Sending article to Groq for deep critical analysis...")
    markdown_report = skeptical_analysis_with_llm(article_text, title)

    print("\n" + markdown_report)  # Print entire Markdown report

if __name__ == "__main__":
    main()


[INFO] Fetching article from https://benjaminspall.com/character-traits/
[INFO] Sending article to Groq for deep critical analysis...

**Critical Analysis Report for: The Ultimate Character Traits List**

## 1. Core Claims

* The article provides a list of positive and negative character traits with definitions.
* The author claims that developing positive character traits is essential for living a happy and fulfilling life.
* The article suggests that understanding and recognizing character traits in oneself and others is crucial for personal growth and success.

## 2. Language & Tone Analysis

* The tone of the article is informative, with a hint of persuasion. The author presents the list of character traits as a valuable resource for personal growth and development.
* The language used is formal and objective, with a focus on providing definitions and explanations of each character trait.
* The author uses phrases such as "we can all agree" and "it's essential" to create a sense of